In [14]:
# Load packages
from selenium import webdriver
import time # used to pause the python for several seconds to wait for the page fully open
import pandas as pd
import numpy as np
import glob
import json

# Fetch and Store the data

In [27]:
# Store the data to a text file
def store_text(text,year,month):
    text_file = open("wunderground_text/Chicago_%d_%d.txt"%(year,month), "w")
    text_file.write(text)
    text_file.close()

# Get the data by a webdriver
def fetch_data(year,month):
    target = "https://www.wunderground.com/history/monthly/KGYY/date/" + str(year) + "-" + str(month)
    dr = webdriver.Chrome()
    dr.get(target)
    time.sleep(4)
    elem = dr.find_element_by_xpath("//div[@class='observation-table ng-star-inserted']")
    text = elem.text
    dr.close()
    store_text(text,year,month)
    return text

In [30]:
# Loop to fetch the data from 2017-9 to 2019-12
years={'2017':[9,10,11,12],'2018':[1,2,3,4,5,6,7,8,9,10,11,12],'2019':[1,2,3,4,5,6,7,8,9,10,11,12]}
for year in list(years.keys()):
    for mon in years[year]:
        fetch_data(int(year),mon)

# Data Cleaning

In [22]:
# Filenames
filenames = glob.glob("wunderground_text/*.txt")
labels = [filename.split("\\")[1].split(".")[0] for filename in filenames]
# Dictionary to store these data
dic = {'Temperature F':{},'Dew Point F':{},'Humidity %':{},
       'Wind Speed mph':{},'Pressure Hg':{},'Precipitation In':[],'Date':[],'Location':[]}

for i in range(len(dic.keys())-3):
    dic[list(dic.keys())[i]]['Max'] = []
    dic[list(dic.keys())[i]]['Avg'] = []
    dic[list(dic.keys())[i]]['Min'] = []
dic

{'Temperature F': {'Max': [], 'Avg': [], 'Min': []},
 'Dew Point F': {'Max': [], 'Avg': [], 'Min': []},
 'Humidity %': {'Max': [], 'Avg': [], 'Min': []},
 'Wind Speed mph': {'Max': [], 'Avg': [], 'Min': []},
 'Pressure Hg': {'Max': [], 'Avg': [], 'Min': []},
 'Precipitation In': [],
 'Date': [],
 'Location': []}

In [23]:
for i,filename in enumerate(filenames):
    file = open(filename,"r")
    file_text = file.read()
    file.close()
    location = labels[i].split('_')[0]
    date = '-'.join(labels[i].split('_')[1:])
    file_text_list = file_text.split("\nMax Avg Min")
    # Each list
    temp = file_text_list[1].split("\n")[1:]
    dew = file_text_list[2].split("\n")[1:]
    humd = file_text_list[3].split("\n")[1:]
    wind = file_text_list[4].split("\n")[1:]
    press = file_text_list[5].split("\nTotal")[0].split("\n")[1:]
    precip = file_text_list[5].split("\nTotal")[1].split("\n")[1:]
    # Get dates
    dates = file_text_list[0].split("\n")[2:]
    for i in range(len(dates)):
        # Dates
        dic['Date'].append(date+'-'+dates[i])
        # Locations
        dic['Location'].append(location)
        # Temperatures
        dic['Temperature F']['Max'].append(float(temp[i].split(" ")[0]))
        dic['Temperature F']['Avg'].append(float(temp[i].split(" ")[1]))
        dic['Temperature F']['Min'].append(float(temp[i].split(" ")[2]))
        # Dew Point
        dic['Dew Point F']['Max'].append(float(dew[i].split(" ")[0]))
        dic['Dew Point F']['Avg'].append(float(dew[i].split(" ")[1]))
        dic['Dew Point F']['Min'].append(float(dew[i].split(" ")[2]))
        # Humidity
        dic['Humidity %']['Max'].append(float(humd[i].split(" ")[0]))
        dic['Humidity %']['Avg'].append(float(humd[i].split(" ")[1]))
        dic['Humidity %']['Min'].append(float(humd[i].split(" ")[2]))
        # Wind Speed
        dic['Wind Speed mph']['Max'].append(float(wind[i].split(" ")[0]))
        dic['Wind Speed mph']['Avg'].append(float(wind[i].split(" ")[1]))
        dic['Wind Speed mph']['Min'].append(float(wind[i].split(" ")[2]))
        # Pressure
        dic['Pressure Hg']['Max'].append(float(press[i].split(" ")[0]))
        dic['Pressure Hg']['Avg'].append(float(press[i].split(" ")[1]))
        dic['Pressure Hg']['Min'].append(float(press[i].split(" ")[2]))
        # Precipitation
        dic['Precipitation In'].append(float(precip[i]))

In [24]:
# Store the data to a json file
json.dump(dic, open("Chicago_2017-09_2019-12.json","w"))

# Store to a Dataframe

In [25]:
dic_dataframe = {}
for key in list(dic.keys()):
    try:
        for sub_key in list(dic[key].keys()):
            dic_dataframe[key+" "+sub_key] = dic[key][sub_key]
    except:
        dic_dataframe[key] = dic[key]

In [31]:
colnames = list(dic_dataframe.keys())[-2:] + list(dic_dataframe.keys())[:-2]
colnames

['Date',
 'Location',
 'Temperature F Max',
 'Temperature F Avg',
 'Temperature F Min',
 'Dew Point F Max',
 'Dew Point F Avg',
 'Dew Point F Min',
 'Humidity % Max',
 'Humidity % Avg',
 'Humidity % Min',
 'Wind Speed mph Max',
 'Wind Speed mph Avg',
 'Wind Speed mph Min',
 'Pressure Hg Max',
 'Pressure Hg Avg',
 'Pressure Hg Min',
 'Precipitation In']

In [33]:
weather_dat = pd.DataFrame.from_dict(dic_dataframe)[colnames]
weather_dat.to_csv('Chicago_2017-09_2019-12.csv',index=False)
weather_dat.head()

,Date,Location,Temperature F Max,Temperature F Avg,Temperature F Min,Dew Point F Max,Dew Point F Avg,Dew Point F Min,Humidity % Max,Humidity % Avg,Humidity % Min,Wind Speed mph Max,Wind Speed mph Avg,Wind Speed mph Min,Pressure Hg Max,Pressure Hg Avg,Pressure Hg Min,Precipitation In
0,2017-10-1,Chicago,70.0,56.8,43.0,43.0,39.0,32.0,90.0,55.0,25.0,14.0,6.8,0.0,29.7,29.6,29.5,0.0
1,2017-10-2,Chicago,82.0,68.2,59.0,50.0,45.0,40.0,63.0,44.8,26.0,16.0,8.8,5.0,29.6,29.3,19.8,0.0
2,2017-10-3,Chicago,81.0,71.2,66.0,67.0,58.6,45.0,84.0,65.5,43.0,14.0,9.3,6.0,29.7,29.6,29.6,0.0
3,2017-10-4,Chicago,75.0,69.0,59.0,68.0,62.7,55.0,94.0,81.0,68.0,14.0,6.0,0.0,29.7,29.6,29.6,0.0
4,2017-10-5,Chicago,73.0,65.3,61.0,61.0,55.7,45.0,96.0,73.5,38.0,9.0,3.8,0.0,29.6,29.5,29.4,0.0
